In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler


data = pd.read_parquet("hf://datasets/BatteRaquette58/airbnb-stock-price/data/airbnb-stock.parquet")
print(data.columns)


X = data.drop('close_last', axis=1)
y = data['close_last'].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)


scaler_y = MinMaxScaler()
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)


X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):

        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2

    def backward(self, X, y, output):
        self.delta2 = (y - output) * self.sigmoid_derivative(output)
        self.dW2 = np.dot(self.a1.T, self.delta2)
        self.db2 = np.sum(self.delta2, axis=0, keepdims=True)
        self.delta1 = np.dot(self.delta2, self.W2.T) * self.sigmoid_derivative(self.a1)
        self.dW1 = np.dot(X.T, self.delta1)
        self.db1 = np.sum(self.delta1, axis=0, keepdims=True)

    def update(self, learning_rate):
        self.W1 += learning_rate * self.dW1
        self.b1 += learning_rate * self.db1
        self.W2 += learning_rate * self.dW2
        self.b2 += learning_rate * self.db2

    def train(self, X, y, epochs, learning_rate):
        for i in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)
            self.update(learning_rate)
            if i % 1000 == 0:
                print(f"Epoch: {i}, Loss: {np.mean(np.square(y - output))}")


input_size = X_train.shape[1]
hidden_size = 5
output_size = 1
nn = NeuralNetwork(input_size, hidden_size, output_size)


epochs = 10000
learning_rate = 0.001
nn.train(X_train, y_train, epochs, learning_rate)


predictions = nn.forward(X_test)

print("Mean Squared Error:", np.mean(np.square(y_test - predictions)))


Index(['date', 'open', 'close_last', 'volume', 'high', 'low'], dtype='object')
Epoch: 0, Loss: 0.03738200592002358
Epoch: 1000, Loss: 0.0024240262169715216
Epoch: 2000, Loss: 0.002016065985044692
Epoch: 3000, Loss: 0.0017821184953151546
Epoch: 4000, Loss: 0.0016096803311478364
Epoch: 5000, Loss: 0.0014720301426002736
Epoch: 6000, Loss: 0.0013604141346877606
Epoch: 7000, Loss: 0.001271115734881305
Epoch: 8000, Loss: 0.0012013191451668253
Epoch: 9000, Loss: 0.001148057594513732
Mean Squared Error: 0.0008468663269520015
